In [1]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings , re , json
from bs4 import BeautifulSoup
from tqdm import tqdm
import matplotlib.pyplot as plt
import multiprocessing as mp
# from DataProcessing import 
import uszipcode , pickle 
from TempFolder.TempFolder import Temp

warnings.filterwarnings('ignore')

In [2]:
with open('nogit\\path','r') as f:
    path_to_csv = f.read()
path_to_processed_csv =path_to_csv + 'SC-csv\\'
Temp.set_path(path_to_csv +'SC-temp\\')
path_to_temp_csv = path_to_csv+'SC-working-folder\\'
colummn_names = ['Description'
                 ,'Title State/Type'
                 ,'Location'
                 ,'null'
                 ,'Auction Date'
                 ,'Actual Cash Value'
                 ,'Repair Cost'
                 ,'Odometer'
                 ,'Prim Damage'
                 ,'Sec Damage'
                 ,'Price Sold or Highest Bid']
path_to_makes = glob.glob(path_to_csv+'Cars\\*')
search = uszipcode.SearchEngine(simple_zipcode=True)
path_to_all_years = list(itertools.chain.from_iterable([glob.glob(path+'\\*') for path in path_to_makes]))
search = uszipcode.SearchEngine(simple_zipcode=True)



In [ ]:
# #%%time
# '''
# Parse the HTML files
# '''

# #
# #Read and parse scraped .html files 
# def process_files(path_to_years__):
#     res_df = pd.DataFrame()
#     #for each make/year folder get a list of all .html files
#     #for path_to__year in tqdm(path_to_years__):
#     for path_to__year in path_to_years__:
#         try:
#             #list of all html
#             path_to_make_years = glob.glob(path_to__year+'/*')

#             #for each .html in the folder
#             for file_path in path_to_make_years:
#             #file_path = path_to_make_years[0]
            
#                 #read file and load to the BS object
#                 with open(file_path, 'r') as f:
#                     fle = f.read()
#                 soup = BeautifulSoup(fle)

#                 rows = []
#                 line_ind = 0 
                
#                 #filter all div elements (div table)
#                 for div in soup.find_all('div',['row']):
#                     row = div.text.strip()
#                     #print(row)
#                     try:
#                         line_dict = {}
#                         i = 0
                        
#                         #clean each element and map to respected column
#                         for line in row.replace('\t','').replace('\xa0','').split('\n'):
#                             line_dict[colummn_names[i]]= line.replace('Location:','').replace('Title State/Type:','').strip()
#                             i+=1
#                         rows.append(line_dict)
#                     except Exception as e:
#                         #print(line_ind,'-',e)
#                         pass
#                     finally:
#                         line_ind+=1

#                 page_df = pd.DataFrame(rows).iloc[3:,:-1]
#                 page_df_ext = page_df.join(page_df['Description'].str.split(' ',expand=True).rename(columns={0:'Year',1:'Make',2:'Model',3:'Model 2',4:'Model 3'}))
#                 page_df_ext = page_df_ext.join(page_df_ext['Location'].str.split('-',expand=True).rename(columns={0:'State',1:'City'}))
#                 page_df_ext['City'] = page_df_ext['City'].str.strip().str.capitalize()
#                 page_df_ext['State'] = page_df_ext['State'].str.strip()
#                 res_df = res_df.append(page_df_ext,ignore_index=True,sort=False)
#                 del rows , page_df_ext , page_df
#         except:
#             continue
#     return res_df


# if __name__ == '__main__':
#     print('# files to concat' , len(path_to_all_years))
#     NCPU = mp.cpu_count()
#     print('Creating pool with ', NCPU,' CPUs')
#     pool = mp.Pool(processes=4)
#     # if not os.path.exists(path_to_processed_csv):
#     #     print('Creating output folder')
#     #     os.makedirs(path_to_processed_csv)
    
#     by_chunk_path_to_all_years = np.array_split(path_to_all_years,len(path_to_all_years) // NCPU )
#     start_ = 0
#     end_ = len(by_chunk_path_to_all_years)
#     for i in range(start_,end_):
#         print('Starting with chunk: ',i,' of ',end_)
#         path_to_all_years_split = np.array_split(by_chunk_path_to_all_years[i],NCPU)
#         ress = pool.map(process_files,path_to_all_years_split)
#         ress_df = pd.concat(ress)
#         print('Chunk ',i,' is done.')
#         ress_df.to_csv(path_to_processed_csv+'sc_'+str(i)+'.csv',index=False)
# #         i+=1
# #         if i ==2:
# #             break
#     pool.close()
#     pool.join()
#     print('all work is done')
    

In [ ]:
#Read parsed files
outputs = glob.glob(path_to_processed_csv+'*')
out_l = []
errs = []
for output in tqdm(outputs):
    try:
#     print(output)
        out_l.append(pd.read_csv(output))
    except:
        errs.append(output)
out_df = pd.concat(out_l,ignore_index=True,sort=False)

if len(errs) >0:
    print('Errors:', errs)

In [ ]:
print(out_df.columns)

row_size = out_df.shape[0]

print('Raw dataset size: ',row_size)


out_df.head(50)


In [ ]:
#clean up wrong values 
out_df = out_df.loc[~out_df['Actual Cash Value'].str.contains('Odometer').fillna(False)]
out_df = out_df.loc[~out_df['Year'].str.contains('Make').fillna(False)]
out_df = out_df.loc[~out_df['Year'].str.contains('Model:').fillna(False)]
out_df = out_df.loc[~out_df['Auction Date'].str.contains('Repair Cost').fillna(False)]


#filter out canadian cars
out_df = out_df.loc[~out_df['Price Sold or Highest Bid'].str.contains('CA').fillna(False)
                               ]

out_df['Auction Date'] = pd.to_datetime(out_df['Auction Date'])

out_df['Auction_Year'] = out_df['Auction Date'].dt.year

In [ ]:
cleaned_size = out_df.shape[0]
print('Cleaned dataset size:',cleaned_size
      ,'\nRows removed: ', row_size - cleaned_size)


In [ ]:
# # out_df.to_csv(path_to_processed_csv+'SC-countymap.csv',index=False)

# # out_df = pd.read_csv(path_to_processed_csv+'SC-countymap.csv')

# from DataProcessing.DataStats import get_df_stats

# get_df_stats(out_df)

In [ ]:
# Combining Prim Damages 
raw_prim_cat = out_df['Prim Damage'].str.lower().unique()
print('Cat count:',raw_prim_cat.shape[0],'\n',raw_prim_cat)

In [ ]:
prim_damage_map = {
 'burn - engine': 'burn'
, 'partial/incomplete r' : 'partial repair'
, 'frame damage reporte' : 'frame damage'

,'unknown' : 'no data'


, np.nan : 'no data'
,'price sold or highest bid' :'no data'
,'rr' : 'no data'

}
out_df['Prim Damage'] = out_df['Prim Damage'].str.lower().replace(prim_damage_map)

In [ ]:
new_prim_cat = out_df['Prim Damage'].str.lower().unique()
print('Cat count:',new_prim_cat.shape[0],'\n',new_prim_cat)

In [ ]:
# out_df = out_df.join(out_df['Location'].str.split('-',expand=True).rename(columns={0:'State',1:'City'}))

In [ ]:
#fix numerical values
def alter_sting(x):    
    try:
        return float(re.sub('[\$,USD,\, ,E,A,N]','',x))
    except:
#         print(x)
        return np.nan


out_df['Actual Cash Value'] = (out_df['Actual Cash Value']
                               #.fillna(-1)
                               .map(alter_sting))
out_df['Repair Cost'] = (out_df['Repair Cost']
#                          .fillna(-1)
                         .map(alter_sting))

out_df['Price Sold or Highest Bid'] = (out_df['Price Sold or Highest Bid']
                                       #.fillna(-1)
                                       .map(alter_sting))
out_df['Year'] = (out_df['Year']
#                   .fillna(-1)
                  .map(float))

out_df = out_df.rename(columns={'Year':'Model_Year'
                               })
                               #.assign(Model_Year = lambda x: int(x))#pd.to_datetime(x['Model_Year']#.astype(int).astype(str))
                      

out_df['City'] = out_df['City'].str.strip().str.capitalize()
out_df['State'] = out_df['State'].str.strip().str.upper()
out_df['Model_short'] = out_df['Model'].astype('str')
out_df['Model'] = out_df['Model'].astype('str') + ' '+out_df['Model 2'].astype('str').fillna('') + ' ' + out_df['Model 3'].fillna('').astype('str')
out_df['Make'] = out_df['Make'].str.strip().str.upper()
out_df['Model'] = out_df['Model'].str.strip().str.upper()
out_df['Model_short'] = out_df['Model_short'].str.strip().str.upper()

#rename variables - add _ where needed

out_df = out_df.rename({'Price Sold or Highest Bid':'Price_Sold_or_Highest_Bid'
                            ,'Repair Cost':'Repair_Cost'
                            ,'Actual Cash Value':'Actual_Cash_Value'
                            ,'Prim Damage':'Prim_Damage'
                            ,'Sec Damage':'Sec_Damage'
                            ,'Title State/Type':'Title_State_Type'
                            ,'Auction Date':'Auction_Date'
                            },axis=1)



# out_df['Auction_Date'] = out_df['Auction_Date'].dt.year
out_df['Model_Year'] = out_df['Model_Year'].astype(int)

#remove columns 

out_df = out_df.drop(columns=['Model 2','Model 3', '2', '5'])


print('New columns names:',out_df.columns)

In [ ]:
out_df.head()

In [ ]:
#format odometer data

#for null model - replace all E  & N values into "No Data" (np.nan)

def odometer_null(x):
    if 'E' in x or 'N' in x:
        return np.nan
    else:
        return alter_sting(x)


    
#create a dictionary for max E value for each model 

odometr_dict = (out_df.loc[out_df['Odometer'].str.contains('A')]
                 .assign(odometer_num = lambda x: x['Odometer'].map(alter_sting)
                        )
                 .groupby('Description')['odometer_num']
                 .max()
                #  [['Odometer','odometer_num']]
                )

#replace E and N values into max of A category
def odometer_replace(x):
    if 'E' in x['Odometer'] or 'N' in x['Odometer']:
        try:
            return odometr_dict[x['Description']]
        except:
            return np.nan
    else:
        return alter_sting(x['Odometer'])


#create null model
out_df = out_df.assign(Odometer_Null = lambda x: x['Odometer'].map(odometer_null))

#create replace model:

out_df = out_df.assign(Odometer_Replace = lambda x: x.apply(odometer_replace,axis=1))


# create Model_Age
# current_year = 2019
out_df['Model_Age'] = (out_df['Auction_Year'] - out_df['Model_Year'] ) + 1 #.dt.days // 365 
# out_df['Model_Year'] = out_df['Model_Year'].dt.year

In [ ]:
out_df.tail()

In [ ]:
%%time
# i = 6
# city = out_df.loc[i,'City']
# state =out_df.loc[i,'State']
# city, state

# geo_found = search.by_city_and_state(city,state)[0]

# geo_found.county , geo_found.state


def find_county(x):
    try:
        return search.by_city_and_state(x[1] , x[0])[0].county
    except:
        return 'None'


locs = out_df['Location'].unique()
locs = locs[~pd.isnull(locs)]

state_city = [l[:2] for l in  list(map(lambda x: x.split(' - '),   locs ) ) #if len(l)>1
             ]





county = list(map(find_county,   state_city ) )

county = [', '.join([l.strip() for l in (county[i].replace('County','')+', '+state_city[i][0]).split(',') ])
 for i in range(len(county))]

loc_lookup = pd.DataFrame({'Location':locs,'County':county}).set_index('Location')['County']

loc_lookup


def map_countries(x):
    try:
        return loc_lookup.loc[x]
    except:
        return 'None'
        
out_df['County'] = out_df['Location'].map(map_countries)
out_df['make_key'] = out_df.assign(Model_Year = lambda x: x['Model_Year'].astype(str))[['Make','Model_short','Model_Year']].agg('_'.join,axis=1)



out_df.head(10)

In [ ]:
# cpi adj original data
base_year = 2010
def cpi_adj_Actual_Cash_Value(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['Auction_Year']]) * x['Actual_Cash_Value'],2)
    except:
        return x['Actual_Cash_Value']


out_df['Actual_Cash_Value_adj'] = out_df.apply(cpi_adj_Actual_Cash_Value
                                            ,axis=1)



def cpi_adj_Price_Sold_or_Highest_Bid(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['Auction_Year']]) * x['Price_Sold_or_Highest_Bid'],2)
    except:
        return x['Price_Sold_or_Highest_Bid']


out_df['Price_Sold_or_Highest_Bid_adj'] = out_df.apply(cpi_adj_Price_Sold_or_Highest_Bid
                                            ,axis=1)

def cpi_adj_Repair_Cost(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['Auction_Year']]) * x['Repair_Cost'],2)
    except:
        return x['Repair_Cost']

out_df['Repair_Cost_adj'] = out_df.apply(cpi_adj_Repair_Cost
                                            ,axis=1)

out_df['make_key'] = out_df.assign(Model_Year = lambda x: x['Model_Year'].astype(str))[['Make','Model_short','Model_Year']].agg('_'.join,axis=1)

#out_df['Make'].value_counts().to_dict()#.to_string()


In [ ]:
out_df.columns

In [3]:
# Temp.save_obj(out_df, 'out_df_working')

out_df = Temp.load_obj('out_df_working')

Object <out_df_working> loaded from: c:\data\Datasets\SC-temp\


In [4]:
'''
END OF MAIN DATASET PREPROCESSING
'''

'\nEND OF MAIN DATASET PREPROCESSING\n'

In [5]:
'''
JOINING WITH DEMOGRAPHIC DATA 
'''


#read the demography data `

path_to_demograpyh =path_to_csv+'SC-working-folder\\demography.csv'

demog = pd.read_csv(path_to_demograpyh)

print('Demographic features:\n' , demog.columns.tolist())



Demographic features:
 ['index', 'Unemp-Pct-2014', 'Unemp-Pct-2015', 'Unemp-Pct-2016', 'Unemp-Pct-2017', 'Unemp-Pct-2018', 'Unemp-Pct-2013', 'Unemp-Pct-2012', 'Unemp-Pct-2011', 'Unemp-Pct-2010', 'Income_adj-1990', 'Income_adj-1991', 'Income_adj-1992', 'Income_adj-1993', 'Income_adj-1994', 'Income_adj-1995', 'Income_adj-1996', 'Income_adj-1997', 'Income_adj-1998', 'Income_adj-1999', 'Income_adj-2000', 'Income_adj-2001', 'Income_adj-2002', 'Income_adj-2003', 'Income_adj-2004', 'Income_adj-2005', 'Income_adj-2006', 'Income_adj-2007', 'Income_adj-2008', 'Income_adj-2009', 'Income_adj-2010', 'Income_adj-2011', 'Income_adj-2012', 'Income_adj-2013', 'Income_adj-2014', 'Income_adj-2015', 'Income_adj-2016', 'Income_adj-2017', 'CENSUS2010', 'CENSUS2011', 'CENSUS2012', 'CENSUS2013', 'CENSUS2014', 'CENSUS2015', 'CENSUS2016', 'CENSUS2017', 'CENSUS2018', 'GRNDTOT_2009', 'GRNDTOT_2010', 'GRNDTOT_2011', 'GRNDTOT_2012', 'GRNDTOT_2013', 'GRNDTOT_2014', 'GRNDTOT_2016', 'CPOPARST_2009', 'CPOPARST_2010', '

In [6]:

#read carfax data
path_to_carfax =path_to_csv+'SC-carfax\\processed\\MSRP.csv'

carfax = pd.read_csv(path_to_carfax,index_col=0) 

#CPI Data
path_to_cpi = path_to_csv+'US-CPI\\SeriesReport-20191118124049_549fff.xlsx'
cpi_df = pd.read_excel(path_to_cpi,skiprows=11,index_col=0)['Annual'][:-2]; cpi_df


#read carfax data
path_to_carfax =path_to_csv+'SC-carfax\\processed\\MSRP.csv'

carfax = pd.read_csv(path_to_carfax,index_col=0) 
carfax['make'] = carfax['make'].str.upper()
carfax['model'] = carfax['model'].str.upper()
carfax['year'] = carfax['year'].astype(str)

carfax['make_key'] = carfax[['make','model','year']].agg('_'.join,axis=1)


carfax['Original_MSRP_mean'] =carfax[['Original_MSRP_low','Original_MSRP_high']].mean(axis=1)

#CPI adjusted income 
def cpi_adj_Original_MSRP_mean(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['year']]) * x['Original_MSRP_mean'],2)
    except:
        return x['Original_MSRP_mean']


carfax['Original_MSRP_mean_adj'] = carfax.apply(cpi_adj_Original_MSRP_mean
                                            ,axis=1)


#carfax['make'].value_counts().to_dict()#.to_string()
carfax.head()


,make,model,year,Original_MSRP_low,Original_MSRP_high,Original_MSRP_mean,Original_MSRP_mean_adj,make_key
0,ACURA,CL,2001,28460.0,30810.0,29635.0,29635.0,ACURA_CL_2001
1,ACURA,CL,2002,28530.0,32880.0,30705.0,30705.0,ACURA_CL_2002
2,ACURA,CL,2003,28700.0,31050.0,29875.0,29875.0,ACURA_CL_2003
3,ACURA,ILX,2013,25900.0,34400.0,30150.0,30150.0,ACURA_ILX_2013
4,ACURA,ILX,2014,26900.0,34600.0,30750.0,30750.0,ACURA_ILX_2014


In [7]:
# join original and carfex
out_df_carfix = pd.merge(out_df, carfax,  how='inner', on ='make_key').drop(columns='make_key')
#out_df_carfix['County']

out_df_carfix.head()

,Actual_Cash_Value,Auction_Date,Description,Location,Odometer,Price_Sold_or_Highest_Bid,Prim_Damage,Repair_Cost,Sec_Damage,Title_State_Type,...,Actual_Cash_Value_adj,Price_Sold_or_Highest_Bid_adj,Repair_Cost_adj,make,model,year,Original_MSRP_low,Original_MSRP_high,Original_MSRP_mean,Original_MSRP_mean_adj
0,NaN,2019-10-21,2004 Oldsmobile ALERO GL,MD - BALTIMORE,151441 A,550.0,normal wear,NaN,Minor dent/scratches,MD CT,...,NaN,550.0,NaN,OLDSMOBILE,ALERO,2004,19110.0,23960.0,21535.0,21535.0
1,1304.0,2019-10-14,2004 Oldsmobile ALERO GL,TN - NASHVILLE,213655 E,800.0,front end,NaN,Rear end,TN CT,...,1304.0,800.0,NaN,OLDSMOBILE,ALERO,2004,19110.0,23960.0,21535.0,21535.0
2,2550.0,2019-09-30,2004 Oldsmobile ALERO GL,AZ - PHOENIX,0 N,400.0,burn,NaN,Minor dent/scratches,AZ CT,...,2550.0,400.0,NaN,OLDSMOBILE,ALERO,2004,19110.0,23960.0,21535.0,21535.0
3,2325.0,2019-09-19,2004 Oldsmobile ALERO GL,CA - SAN BERNARDINO,241435 A,200.0,front end,5712.0,Side,CA SC,...,2325.0,200.0,5712.0,OLDSMOBILE,ALERO,2004,19110.0,23960.0,21535.0,21535.0
4,2107.0,2018-07-17,2004 Oldsmobile ALERO GL,LA - BATON ROUGE,0 E,250.0,front end,2107.0,NaN,LA ST,...,2107.0,250.0,2107.0,OLDSMOBILE,ALERO,2004,19110.0,23960.0,21535.0,21535.0


In [8]:
# with open('var/final_columns.json') as f:
#     final_columns = json.load(f)['final_columns']

car_demo_joined = (pd.merge(out_df_carfix, demog,left_on='County',right_on='index',how='left')
                    .drop(columns=['Location','Description','Odometer','Sec_Damage','Title_State_Type','State', 'City','make', 'model','year'])					
                    )
				


print(car_demo_joined.shape, car_demo_joined.head(),'\n\n' ,car_demo_joined.columns)




(516218, 82)    Actual_Cash_Value Auction_Date  Price_Sold_or_Highest_Bid  Prim_Damage  \
0                NaN   2019-10-21                      550.0  normal wear   
1             1304.0   2019-10-14                      800.0    front end   
2             2550.0   2019-09-30                      400.0         burn   
3             2325.0   2019-09-19                      200.0    front end   
4             2107.0   2018-07-17                      250.0    front end   

   Repair_Cost  Model_Year        Make     Model  Auction_Year Model_short  \
0          NaN        2004  OLDSMOBILE  ALERO GL          2019       ALERO   
1          NaN        2004  OLDSMOBILE  ALERO GL          2019       ALERO   
2          NaN        2004  OLDSMOBILE  ALERO GL          2019       ALERO   
3       5712.0        2004  OLDSMOBILE  ALERO GL          2019       ALERO   
4       2107.0        2004  OLDSMOBILE  ALERO GL          2018       ALERO   

   ...  GRNDTOT_2013  GRNDTOT_2014  GRNDTOT_2016 CPOPAR

In [9]:
#create master demographic field 

# 1 Unemp-Pct

unemp_columns  = [col for col in car_demo_joined.columns if 'Unemp-Pct' in col]
print('working with ' ,unemp_columns)
def fetch_unemp_null(x):
    ''' Return the actual on the action date unemployment, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Unemp-Pct-{auction_y}']
    except:
        return

car_demo_joined['Unemp_pct_null'] = car_demo_joined.apply(fetch_unemp_null, axis=1)

unemp_available = np.array([2014,2015,2016,2017,2018,2013,2012,2011,2010])
def fetch_unemp_closest(x):
    ''' Return the actual on the action date unemployment, if none, return abs closest'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Unemp-Pct-{auction_y}']
    except:
        closest_year_ix = np.abs( unemp_available - x['Auction_Year']).argmin()
        closest_year = unemp_available[closest_year_ix]
        return x[f'Unemp-Pct-{closest_year}']

car_demo_joined['Unemp_pct_closest'] = car_demo_joined.apply(fetch_unemp_closest, axis=1)


working with  ['Unemp-Pct-2014', 'Unemp-Pct-2015', 'Unemp-Pct-2016', 'Unemp-Pct-2017', 'Unemp-Pct-2018', 'Unemp-Pct-2013', 'Unemp-Pct-2012', 'Unemp-Pct-2011', 'Unemp-Pct-2010']


In [10]:
# 2 Income_adj
inc_columns  = [col for col in car_demo_joined.columns if 'Income_adj' in col]
print('working with ' ,inc_columns)

def fetch_inc_null(x):
    ''' Return the actual on the action date income, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Income_adj-{auction_y}']
    except:
        return

car_demo_joined['Income_adj_null'] = car_demo_joined.apply(fetch_inc_null, axis=1)


income_available = np.arange(1990,2018)
def fetch_inc_closest(x):
    ''' Return the actual on the action date income, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Income_adj-{auction_y}']
    except:
        closest_year_ix = np.abs( income_available - x['Auction_Year']).argmin()
        closest_year = income_available[closest_year_ix]
        return x[f'Income_adj-{closest_year}']

car_demo_joined['Income_adj_closest'] = car_demo_joined.apply(fetch_inc_closest, axis=1)


working with  ['Income_adj-1990', 'Income_adj-1991', 'Income_adj-1992', 'Income_adj-1993', 'Income_adj-1994', 'Income_adj-1995', 'Income_adj-1996', 'Income_adj-1997', 'Income_adj-1998', 'Income_adj-1999', 'Income_adj-2000', 'Income_adj-2001', 'Income_adj-2002', 'Income_adj-2003', 'Income_adj-2004', 'Income_adj-2005', 'Income_adj-2006', 'Income_adj-2007', 'Income_adj-2008', 'Income_adj-2009', 'Income_adj-2010', 'Income_adj-2011', 'Income_adj-2012', 'Income_adj-2013', 'Income_adj-2014', 'Income_adj-2015', 'Income_adj-2016', 'Income_adj-2017']


In [11]:
# 3 CENSUS
CENSUS_columns  = [col for col in car_demo_joined.columns if 'CENSUS' in col]
print('working with ' ,CENSUS_columns)

def CENSUS_null(x):
    ''' Return the actual on the action date CENSUS, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'CENSUS{auction_y}']
    except:
        return

car_demo_joined['CENSUS_null'] = car_demo_joined.apply(CENSUS_null, axis=1)


CENSUS_available = np.arange(2010,2019)
def CENSUS_closest(x):
    ''' Return the actual on the action date CENSUS, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'CENSUS{auction_y}']
    except:
        closest_year_ix = np.abs( CENSUS_available - x['Auction_Year']).argmin()
        closest_year = CENSUS_available[closest_year_ix]
        return x[f'CENSUS{closest_year}']

car_demo_joined['CENSUS_closest'] = car_demo_joined.apply(CENSUS_closest, axis=1)


working with  ['CENSUS2010', 'CENSUS2011', 'CENSUS2012', 'CENSUS2013', 'CENSUS2014', 'CENSUS2015', 'CENSUS2016', 'CENSUS2017', 'CENSUS2018']


In [12]:
# 4 - GRNDTOT
GRNDTOT_columns  = [col for col in car_demo_joined.columns if 'GRNDTOT' in col]
print('working with ' ,GRNDTOT_columns)

def GRNDTOT_null(x):
    ''' Return the actual on the action date GRNDTOT, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'GRNDTOT_{auction_y}']
    except:
        return

car_demo_joined['GRNDTOT_null'] = car_demo_joined.apply(GRNDTOT_null, axis=1)


GRNDTOT_available = np.array([2009,2010,2011,2012,2013,2014,2016])
def GRNDTOT_closest(x):
    ''' Return the actual on the action date GRNDTOT, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'GRNDTOT_{auction_y}']
    except:
        closest_year_ix = np.abs( GRNDTOT_available - x['Auction_Year']).argmin()
        closest_year = GRNDTOT_available[closest_year_ix]
        return x[f'GRNDTOT_{closest_year}']

car_demo_joined['GRNDTOT_closest'] = car_demo_joined.apply(GRNDTOT_closest, axis=1)


working with  ['GRNDTOT_2009', 'GRNDTOT_2010', 'GRNDTOT_2011', 'GRNDTOT_2012', 'GRNDTOT_2013', 'GRNDTOT_2014', 'GRNDTOT_2016']


In [13]:
# 5 - CPOPARST
CPOPARST_columns  = [col for col in car_demo_joined.columns if 'CPOPARST' in col]
print('working with ' ,GRNDTOT_columns)

def CPOPARST_null(x):
    ''' Return the actual on the action date CPOPARST, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'CPOPARST_{auction_y}']
    except:
        return

car_demo_joined['CPOPARST_null'] = car_demo_joined.apply(CPOPARST_null, axis=1)


CPOPARST_available = np.array([2009,2010,2011,2012,2013,2014,2016])
def CPOPARST_closest(x):
    ''' Return the actual on the action date CPOPARST, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'CPOPARST_{auction_y}']
    except:
        closest_year_ix = np.abs( CPOPARST_available - x['Auction_Year']).argmin()
        closest_year = CPOPARST_available[closest_year_ix]
        return x[f'CPOPARST_{closest_year}']

car_demo_joined['CPOPARST_closest'] = car_demo_joined.apply(CPOPARST_closest, axis=1)


working with  ['GRNDTOT_2009', 'GRNDTOT_2010', 'GRNDTOT_2011', 'GRNDTOT_2012', 'GRNDTOT_2013', 'GRNDTOT_2014', 'GRNDTOT_2016']


In [15]:
#remove yearly demographic data 
col_to_rem = [unemp_columns,inc_columns ,CENSUS_columns,GRNDTOT_columns,CPOPARST_columns]
from itertools import chain
col_to_rem_fl = list(chain.from_iterable(col_to_rem))
#col_to_rem#[np.array(i) for i in col_to_rem]

car_demo_joined = car_demo_joined.drop(columns=col_to_rem_fl)

In [18]:
car_demo_joined.loc[1].to_dict()

{'Actual_Cash_Value': 1304.0,
 'Auction_Date': Timestamp('2019-10-14 00:00:00'),
 'Price_Sold_or_Highest_Bid': 800.0,
 'Prim_Damage': 'front end',
 'Repair_Cost': nan,
 'Model_Year': 2004,
 'Make': 'OLDSMOBILE',
 'Model': 'ALERO GL',
 'Auction_Year': 2019,
 'Model_short': 'ALERO',
 'Odometer_Null': nan,
 'Odometer_Replace': 297038.0,
 'Model_Age': 16,
 'County': 'Davidson, TN',
 'Actual_Cash_Value_adj': 1304.0,
 'Price_Sold_or_Highest_Bid_adj': 800.0,
 'Repair_Cost_adj': nan,
 'Original_MSRP_low': 19110.0,
 'Original_MSRP_high': 23960.0,
 'Original_MSRP_mean': 21535.0,
 'Original_MSRP_mean_adj': 21535.0,
 'index': 'Davidson, TN',
 'Unemp_pct_null': nan,
 'Unemp_pct_closest': 2.6,
 'Income_adj_null': nan,
 'Income_adj_closest': 56100.136781984336,
 'CENSUS_null': nan,
 'CENSUS_closest': 692587.0,
 'GRNDTOT_null': nan,
 'GRNDTOT_closest': 45438.0,
 'CPOPARST_null': nan,
 'CPOPARST_closest': 687373.0}

In [ ]:
car_demo_joined.to_csv(path_to_temp_csv+'joint_working_df.csv')
Temp.save_obj(car_demo_joined,'car_demo_joined')

In [ ]:
'''
ANALYSIS
'''

In [ ]:
del out_df
out_df = Temp.load_obj('out_df_working')
# out_df = Temp.load_obj('car_demo_joined')

In [ ]:
out_df.groupby('Make')['Model_Year'].count().sort_values(ascending=False)[:60]

In [ ]:
#explore outlyers for 
var_to_plot ='Model_Age'
fig, ax = plt.subplots(1,2,figsize=(7,5))
out_df[var_to_plot].plot.box(ax=ax[0])
out_df[var_to_plot].plot.box(ax=ax[1],showfliers =False)


fig.suptitle(var_to_plot)

In [ ]:
# filterd_acv = out_df[var_to_plot]
# filterd_acv_round = filterd_acv


# filterd_acv_round_vc = filterd_acv_round.value_counts().to_frame().reset_index().rename({'index':'level' , var_to_plot:'count'}, axis=1)
# filterd_acv_round_vc.to_csv('csv/outly.csv', index=False)

In [ ]:
#histogram

s = 20
e = 80
#display modes of the distribution 
try:
    filterd_acv = out_df[var_to_plot].loc[(out_df[var_to_plot] >s )& (out_df[var_to_plot] <e)]

    print('Counts:\n#>100k:'
                ,len(out_df[var_to_plot].loc[out_df[var_to_plot]>=1e5]))

except:
    filterd_acv = out_df[var_to_plot]




n = 1
filterd_acv_round = (filterd_acv // n ) * n

filterd_acv.plot.hist(title= var_to_plot+f' (between {s} and {e})',color='k', alpha=.4)

filterd_acv_round_vc = filterd_acv_round.value_counts().to_frame().reset_index().rename({'index':'level' , var_to_plot:'count'}, axis=1)
filterd_acv_round_vc.to_csv('csv/outly.csv', index=False)
# filterd_acv_round_vc

In [ ]:


#plot a histogram of observed cash_values
fig= plt.figure()
ax = filterd_acv.plot.hist()
ax.set(title = 'Histogram for Actual_Cash_Value < 10k');





In [ ]:
#fit  normat distribution

from scipy import stats

x = np.linspace(0,filterd_acv.max(),len(filterd_acv))
mu , std = stats.norm.fit(filterd_acv.values)

ax = filterd_acv.plot.hist(normed=True)
plt.plot(x, stats.norm.pdf(x , mu, std))
ax.set(title = 'Histogram for Actual_Cash_Value < 10k');
print('K-S p-value:', stats.kstest(filterd_acv.values , 'norm')[1])



In [ ]:

out_df['Model_Age'].describe().astype(int)

In [ ]:
#model_age
#check model age statsВ

out_df.loc[out_df['Model_Age'].loc[out_df['Model_Age']<0].index]

In [ ]:

numerical_fields =  ['Actual_Cash_Value'
               ,'Auction_Year'
       #'Odometer'
       , 'Price_Sold_or_Highest_Bid'
       , 'Repair_Cost'
        ,'Odometer_Null','Odometer_Replace'
        ,'Model_Age'
        ]

categorical_variables = ['Prim_Damage','Make','Model'
]


In [ ]:
#categories for all categorical variables 

# categorial_variables = ['Prim_Damage'
#                         ,'Sec_Damage'
#                         ,'Make'
#                         ,'Model' # - to many categories >2000
#
                        #  ]  
out_df[categorical_variables]  =out_df[categorical_variables].astype('category')

print(out_df[categorical_variables].describe().to_string(),'\n\n')

cat_df = {}
for col in categorical_variables:
    print(col, ' - ', out_df[col].cat.categories.tolist(),'\n')
    cat_df[col] = out_df[col].cat.categories.tolist()


In [ ]:
model_year_count = out_df['Model_Year'].value_counts()
print('total count:',out_df.shape[0],'\ntotal count (model_year >= 2000):', out_df.loc[out_df.Model_Year >=2000].shape[0])
model_year_count.head(20)Z

In [ ]:
out_df.columns

In [ ]:
out_df_describe = out_df[numerical_fields].describe(percentiles =[.01,.25, .5, .75,.99]).T

out_df_describe.to_csv('csv/summary_statistics.csv')

out_df_describe

In [ ]:
#Correlation Analysis
corr_table = out_df[numerical_fields].corr()

corr_table.to_csv('csv/correlation.csv')

corr_table

In [ ]:
numerical_fields_bins = {'Actual_Cash_Value' : [],
'Price_Sold_or_Highest_Bid': [],
'Repair_Cost': [],
'Odometer_Null': [],
'Odometer_Replace': [],
'Model_Age': }

In [ ]:
#histomgram for conitnuous variables
fig , axs = plt.subplots(len(numerical_fields) ,2, figsize=(12,29),sharex=False)


i=0
for var in numerical_fields:
    # hist_df_lb ,hist_df_hb = out_df[var].loc[lambda x: x <=3e4] , out_df[var].loc[lambda x: x >=1e4]
    df_to_print = (out_df[var] // 100).value_counts().to_frame().sort_values(by=var)
    df_to_print.to_csv(f'csv/{var}_hist.csv')
    print(df_to_print)
    # hist_df_lb.plot.hist(bins=10, ax= axs[i][0])
    # hist_df_hb.plot.hist(bins=10, ax= axs[i][1])
    
    # axs[i][0].set(title = var + ' <100k bin')
    # axs[i][1].set(title = var + ' >100k bin')
    i+=1

In [ ]:
#market size

market_size = out_df.groupby(out_df['Auction_Date'].dt.year)['Actual_Cash_Value'].sum()

market_size =  market_size / 1000000

print(market_size.to_string())
plt.figure(figsize=(10,4))
ax = market_size.plot.bar()
ax.set(title='Market Size', ylabel='$, millions');

In [ ]:
out_df[categorical_variables]

from DataProcessing.DataStats import get_df_stats

get_df_stats(out_df[categorical_variables])

In [ ]:
out_df.columns

In [ ]:
#value distribution for Description 

desc_val_dist = out_df['Description'].value_counts()

desc_val_dist[:30]

In [ ]:
#value distribution for Prim Damage 

prim_val_dist = out_df['Prim_Damage'].value_counts()

prim_val_dist[:30]

In [ ]:
#summary statistics conditions on Prim_Damage

cond_on_primd = out_df.groupby('Prim_Damage').agg(['mean','var']).stack().sort_index(level=1)
cond_on_primd.index = ['{} - {}'.format(j, i) for i, j in cond_on_primd.index]
cond_on_primd

In [ ]:
#summary statistics conditions on Make

cond_on_make = out_df.groupby('Make').agg(['mean','var']).stack().sort_index(level=1)
cond_on_make.index = ['{} - {}'.format(j, i) for i, j in cond_on_make.index]
cond_on_make.to_csv('csv/temp.csv')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(cond_on_make)

In [ ]:
by_couny = (car_demo_joined.reset_index()
             .groupby('County')['index']
             .count().sort_values(ascending=False)
           )

print(by_couny.to_string())

In [ ]:
plt.figure()
ax = by_couny[:20].plot.bar()
ax.set(title='Top 20 Counties by # records',ylabel='# observations');

In [ ]:
#categories for all categorical variables 
# categorial_variables = ['Prim_Damage'
#                         ,'Sec_Damage'
#                         ,'Make'
#                         ,'Model' # - to many categories >2000
#
                        #  ]  
out_df[categorial_variables]  =out_df[categorial_variables].astype('category')

print(out_df[categorial_variables].describe().to_string(),'\n\n')

cat_df = {}
for col in categorial_variables:
    print(col, ' - ', out_df[col].cat.categories.tolist(),'\n')
    cat_df[col] = out_df[col].cat.categories.tolist()
